<a href="https://colab.research.google.com/github/GeorgeWilliam1999/XAI-Project/blob/main/Quantum_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Next points

Lime on paramters (J,h) in place of ground state

Does the circuit mimic the magnetisation observable? - No

How can you use Lime to find a circuit that mimics an observable? - Not yet


Kitev model on a square lattice hamiltonian - Hamitonian 

Add RF to end



Quantum network and calssical network

4 training sizes

1: quantum network + classical

    - Neural network?

2: pure classical

3: pure quantum

##### SWAP GATE CODE

In [ ]:
# def SWAP(qubits,state):
#     Q_C = QuantumCircuit(qubits*2+1)
#     Q_C.add_H_gate(0)
#     for i in range(qubits):
#         testgate = gate.SWAP(i+1,i+qubits+1)
#         test_gate = gate.to_matrix_gate(testgate)
#         test_gate.add_control_qubit(nq,0)
#         Q_C.add_gate(test_gate)
#     Q_C.add_H_gate(0)
#     Q_C.update_quantum_state(state)
#     return state

In [ ]:
# import numpy as np
# from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

# num_qubits = 6
# circuit1 = QuantumCircuit(num_qubits)
# for i in range(num_qubits):
#     circuit1.x(i)

# circuit2 = QuantumCircuit(num_qubits)
# for i in range(num_qubits):
#     circuit2.id(i)
# circ1 = [i for i in range(1,num_qubits+1)]
# circ2 = [i for i in range(num_qubits+1,2num_qubits+1)]
# swap_test_circuit = QuantumCircuit(2num_qubits + 1,1)
# swap_test_circuit.compose(circuit1, qubits=circ1, inplace=True )
# swap_test_circuit.compose(circuit2, qubits=circ2, inplace=True )
# swap_test_circuit.h(0)
# for i in range(num_qubits):
#     swap_test_circuit.cswap(0,i+1,i+num_qubits+1)
# swap_test_circuit.h(0)
# swap_test_circuit.measure([0],[0])
# print(swap_test_circuit.draw('text'))

## Packages


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install qulacs
!pip install lime
!pip install -q --upgrade ipython
!pip install -q --upgrade ipykernel
!pip install openFermion
#!pip install numba
#!pip install kwant

In [ ]:
import qulacs
from qulacs import ParametricQuantumCircuit
from qulacs import QuantumState
from qulacs.gate import X, RX, RY, RZ, DenseMatrix, Z, to_matrix_gate
from qulacs import Observable
from qulacs import QuantumCircuit

from qulacs.gate import Identity # Identity matrix
from qulacs.gate import X, Y, Z     # Pauli
from qulacs.gate import H, S, Sdag, sqrtX, sqrtXdag, sqrtY, sqrtYdag # クリフォード
from qulacs.gate import T, Tdag # T gate
from qulacs.gate import P0, P1 # Projection to 0,1 (not normalized)

import numpy as np
from numpy import linalg as LA
import seaborn as sns
import pandas as pd 


import sklearn
import sklearn.datasets
import sklearn.ensemble
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from scipy.optimize import minimize #BFGS works well/cobyla
from scipy.sparse import linalg as linalg
import scipy

from matplotlib import pyplot as plt

import lime
import lime.lime_tabular

import numpy as np
#from scipy.sparse import linalg

import cirq
import openfermion as of
from openfermion import QubitOperator
from openfermion import FermionOperator
from openfermion.linalg import qubit_operator_sparse,get_sparse_operator
#import openfermionpyscf as ofpyscf

#Parallelisation
import numba
from numba import njit, jit, prange

#Ad hoc
import math
from math import sqrt

#https://www.pythonpool.com/pythons-itertool-product/
import itertools as it

In [ ]:
#Ising model time evloution

def n_q_spin_chain_evol(n):

  # Number of qubits : n
  # Theta : set of parameters for PQC

  angle = 0

  # construct parametric quantum circuit with random rotation
  circuit = ParametricQuantumCircuit(n)
  for i in range(n):
    circuit.add_H_gate(i)
  for i in range(n):
    circuit.add_parametric_RZ_gate(i,angle)
    circuit.add_parametric_RZ_gate((i+1)%n,angle)
    circuit.add_H_gate(i+1)
    circuit.add_CNOT_gate(i,i+1) #Needs to be rotX?
    circuit.add_H_gate(i+1)
    circuit.add_parametric_RX_gate(i,angle)
  for i in range(n):
    circuit.add_H_gate(i)

  return circuit

## Ising Model Energy Spectrums

In [ ]:
#Transverse field Ising Hamiltonian:

def T_F_I(n,J,h,periodic_bc = False):
  H = QubitOperator()
  max_index = n if periodic_bc else n - 1
  for i in range(max_index):
      next_neighbour_index = 0 if i == n - 1 and periodic_bc else i + 1
      H += QubitOperator(f"Z{i} Z{next_neighbour_index}", -J)
  for i in range(n):
    H += QubitOperator(f"X{i}", -h) #I must be removed
    H += QubitOperator(f"Z{i}", 0.01)
  return H

In [ ]:
#Longatudinal field Ising Hamiltonian:

def L_F_I(n,J,h,periodic_bc = False):
  H = QubitOperator()
  max_index = n if periodic_bc else n - 1
  for i in range(max_index):
      next_neighbour_index = 0 if i == n - 1 and periodic_bc else i + 1
      H += QubitOperator(f"Z{i} Z{next_neighbour_index}", -J)
  for i in range(n):
    H += QubitOperator(f"X{i}", 0.01) #I must be removed
    H += QubitOperator(f"Z{i}", -h)
  return H

In [ ]:
H_weak = T_F_I(8,1,0.1,periodic_bc = False)
H_weak = qubit_operator_sparse(H_weak).todense()
H_strong = T_F_I(8,1,1,periodic_bc = False)
H_strong = qubit_operator_sparse(H_strong).todense()

val_weak,vec_weak = LA.eig(H_weak)
val_strong,vec_strong = LA.eig(H_strong)
val_weak = np.sort(val_weak)
val_strong = np.sort(val_strong)

plt.plot(np.linspace(0,len(val_weak),len(val_weak)),val_weak)
plt.plot(np.linspace(0,len(val_weak),len(val_weak)),val_strong)
plt.legend(['Weak transverse field (h=0.1)','Strong transverse field (h=1)'])
plt.title('Transverse Ising Spectrum')
plt.xlabel('Eigen state number')
plt.ylabel('Energy')

In [ ]:
H_weak = L_F_I(8,-1,0.1,periodic_bc = False)
H_weak = qubit_operator_sparse(H_weak).todense()
H_strong = L_F_I(8,-1,1,periodic_bc = False)
H_strong = qubit_operator_sparse(H_strong).todense()

val_weak,vec_weak = LA.eig(H_weak)
val_strong,vec_strong = LA.eig(H_strong)
val_weak = np.sort(val_weak)
val_strong = np.sort(val_strong)

plt.plot(np.linspace(0,len(val_weak),len(val_weak)),val_weak)
plt.plot(np.linspace(0,len(val_weak),len(val_weak)),val_strong)
plt.legend(['Weak longitudinal field (h=0.1)','Strong transverse field (h=1)'])
plt.title('Longitudinal Ising Spectrum')
plt.xlabel('Eigen state number')
plt.ylabel('Energy')

## Magnetisation 

In [ ]:
def magnetisation_expectation_X(n,H):
  ## Find the ground state of the hamiltonian 
  E_vals,E_vecs = linalg.eigs(H) #sparse Hamiltonian
  gs = E_vecs[:,np.argmin(E_vals)]

  ge = np.amin(E_vals)

  ## Define the magnetisation
  M = QubitOperator()
  for i in range(n):
    M += QubitOperator(f"X{i}", 1) 
  M = 1/n * qubit_operator_sparse(M).todense().real

  ## Compute the ground state magnetisation
  #print(gs.shape,M.shape)
  m_0 = np.conjugate(gs.T) @ M @ gs

  return m_0,gs,ge

def magnetisation_expectation_Y(n,H):
  ## Find the ground state of the hamiltonian 
  E_vals,E_vecs = linalg.eigs(H) #sparse Hamiltonian
  gs = E_vecs[:,np.argmin(E_vals)]

  ge = np.amin(E_vals)

  ## Define the magnetisation
  M = QubitOperator()
  for i in range(n):
    M += QubitOperator(f"Y{i}", 1) 
  M = 1/n * qubit_operator_sparse(M).todense().real

  ## Compute the ground state magnetisation
  #print(gs.shape,M.shape)
  m_0 = np.conjugate(gs.T) @ M @ gs

  return m_0,gs,ge

def magnetisation_expectation_Z(n,H):
  ## Find the ground state of the hamiltonian 
  E_vals,E_vecs = linalg.eigs(H) #sparse Hamiltonian
  gs = E_vecs[:,np.argmin(E_vals)]

  ge = np.amin(E_vals)

  ## Define the magnetisation
  M = QubitOperator()
  for i in range(n):
    M += QubitOperator(f"Z{i}", 1) 
  M = 1/n * qubit_operator_sparse(M).todense().real

  ## Compute the ground state magnetisation
  #print(gs.shape,M.shape)
  m_0 = np.conjugate(gs.T) @ M @ gs

  return m_0,gs,ge

In [ ]:
n = 4
field_constants = np.linspace(-5,5,50)
magnetisations = np.zeros(len(field_constants))
for i in range(len(field_constants)):
  H_dummy = qubit_operator_sparse(L_F_I(n,1,field_constants[i],periodic_bc = False)).todense().real
  m,g,e = magnetisation_expectation_Z(n,H_dummy)
  magnetisations[i] = m[0,0]

In [ ]:
plt.plot(field_constants,magnetisations)
#plt.xscale('log')
plt.legend(['Magnetisation'])
plt.title('Magnetisations of various field strengths')
plt.xlabel('Relative transverse field strength')
plt.ylabel('Magnetisation')
#plt.axvline(5)

## Classification Object

In [ ]:
class MLP_2():

  def __init__(self,n,U,Embed):
    self.U = U(n)
    self.E = Embed
    self.theta = np.random.rand(self.U.get_parameter_count())
    self.classes = 0
    self.expectation_values = 0
    #self.n = n #good practice!


  #Determine optimal parameter vector theta
  def train(self,X,y):
    '''print(self.theta)'''

    #Set initial theta to a a random set of numbers
    theta_0 = self.theta

    #Determine the number of classes
    classes = np.unique(y)
    self.classes = classes

    #Compute the expectation values of a data point
    def find_expectations(theta):  
      Expectations = np.zeros(X.shape[0])
      circuit = self.U

      j = 0
      for row in np.array(X):
        for i in range(circuit.get_parameter_count()):
          circuit.set_parameter(i,theta[i])
        #Define and embed data point, x is an element of the normalised data set
        point = row
        #point = np.kron(np.array([1,0]),np.kron(np.array([1,0]),row))
        state = self.E(point)
        circuit.update_quantum_state(state)
        # for k in range(n):
          
        #   obs = Observable(n)
        #   obs.add_operator(1.,'Z {}'.format(k))
        #   exp = obs.get_expectation_value(state)
          #print(k,exp)
        #obs = Observable(n + 2)
        obs = Observable(n)
        obs.add_operator(1.,'Z {}'.format(n-1))
       # print(n)
        exp = obs.get_expectation_value(state)
        Expectations[j] = ((exp + 1)/2)*max(classes)
        j += 1
        
      return Expectations

    def MSE(theta):
      #get predictions

      Expectations = find_expectations(theta)
      #Simple MSE example
      
      #Expectation values
      L = ((Expectations - y)**2).mean()
      #L =  -((y*np.log(Expectations) + (1 - y)*np.log(1-Expectations))).mean()
      print('The loss is ', L)
      #print('The expectation is ', Expectations)
      return L

    result = minimize(MSE, theta_0, method='COBYLA')
    self.theta = result.x
    '''print(theta_0, self.theta)'''
    return self.theta

  def predict(self,X):
    classes = self.classes
    #Compute the expectation values of a data point
    
    def find_expectations(theta):  
      Expectations = np.zeros(X.shape[0])
      circuit = self.U

      j = 0
      for row in np.array(X):
        for i in range(circuit.get_parameter_count()):
          circuit.set_parameter(i,theta[i])
        #Define and embed data point, x is an element of the normalised data set
        point = row
        #point = np.kron(np.array([1,0]),np.kron(np.array([1,0]),row))
        state = self.E(point)
        circuit.update_quantum_state(state)
        # for k in range(n):
          
        #   obs = Observable(n)
        #   obs.add_operator(1.,'Z {}'.format(k))
        #   exp = obs.get_expectation_value(state)
          #print(k,exp)
        #obs = Observable(n + 2)
        obs = Observable(n)
        obs.add_operator(1.,'Z {}'.format(n-1))
       # print(n)
        exp = obs.get_expectation_value(state)
        Expectations[j] = ((exp + 1)/2)*max(classes)
        j += 1
        
      return Expectations
    
    expectations = find_expectations(self.theta)
    self.expectation_values = expectations
    '''print(self.expectation_values)'''
    return expectations

  #Create probabilities for LIME using any classifier that has a method of the 
  #form clssifier.predict() which takes a single point a its argument

  ###Assign probabilities
  def create_probabilities(self,input_X):

    if input_X.ndim == 1:
      input_X = np.reshape(input_X,(1,input_X.shape[0]))
    
    expectation_values = self.predict(input_X)
  
    probabilities = np.zeros((input_X.shape[0],2))

    for i in range(input_X.shape[0]):
      e = expectation_values[i]
      p_0 = 1-e
      p_1 = e
      probabilities[i,:] = np.array([p_0, p_1])
      probabilities[i,:] = probabilities[i,:]/(probabilities[i,:].sum())
      
    return probabilities

  def create_probabilities_Ising_field(self,input_X):

    if input_X.ndim == 1:
      input_X = np.reshape(input_X,(1,input_X.shape[0]))  

    # print('The input field strengths are: ',input_X)
    probabilities = np.zeros((input_X.shape[0],2))

    for i in range(input_X.shape[0]):
      E_vals,E_vecs = linalg.eigs(qubit_operator_sparse(T_F_I(n,J =input_X[i,0],h = input_X[i,1]))) #sparse Hamiltonian
      gs = E_vecs[:,np.argmin(E_vals)]
      # print(gs,gs.shape)
      ge = np.amin(E_vals)
      e = self.predict(np.array([gs]))
      # print('the expectation value is: ', e)
      p_0 = 1-e
      p_1 = e
      probabilities[i,:] = np.reshape(np.array([p_0, p_1]),probabilities[i,:].shape)
      probabilities[i,:] = probabilities[i,:]/(probabilities[i,:].sum())           
      
    return probabilities


#### testing


In [ ]:
# X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(ising_data.iloc[:,0:2**n], ising_data.iloc[:,2**n:2**n+3], test_size=0.7)

# classifier_1 = MLP_2(n,U_conv_6,pointwise_identity_embedding)
# classifier_1.train(X_train_1,y_train_1.iloc[:,1])
# expectations_1 = classifier_1.predict(X_test_1.iloc[:,0:2**n])

# def classify_bi(exp):
#   if exp >= 0.5:
#     pred = 1
#   else:
#     pred = 0
#   return pred

#  #Check the accuracy of the predictions 
# correct_1 = 0

# q_predictions_1 = np.zeros(len(X_test_1.iloc[:,0]))
# for i in range(len(q_predictions_1)):
#   q_predictions_1[i] = classify_bi(expectations_1[i])
#   #print(q_predictions[i])
#   if q_predictions_1[i] == y_test_1.iloc[i,1]:
#     correct_1 += 1

# accuracy_1 = (correct_1/len(q_predictions_1))*100

# print('The binary classification for the Ising Model data set is ', accuracy_1 ,'% accurate ')

# y_test_1.insert(3,'expectation values',expectations_1)
# y_test_1.insert(4,'predicted classes',q_predictions_1)

# y_test_1.sort_values('True Magnetisations')

In [ ]:
# classifier = classifier_1
# classifier.create_probabilities_Ising_field(np.array(field_features.iloc[0:5,:]))

In [ ]:
# classifier_1.predict(np.array([X_test_1.iloc[0,0:2**n]]))

In [ ]:
# y_test_1.iloc[0:5,0:2**n]

In [ ]:
# a = np.array([[0,1],[0,2],[0,3]])
# for i in a:
#   print(i)

In [ ]:
# a.shape

In [ ]:
# b = np.array([[0,1]])
# for i in b:
#   print(i)

## Quantum Circuits


In [ ]:
a = np.array([[0,1],[0,2],[0,3]])
for i in a:
  print(i)

In [ ]:
def U_conv_6(d):
  a = 0 #2 ancila qubits
  n = d + a

  #Metadata
  angle = 0
  # construct parametric quantum circuit with random rotation
  circuit = ParametricQuantumCircuit(n)

  # for i in range(n):
  #   circuit.add_H_gate(i)

  for i in range(math.ceil(d/2)+1):
    print('Pass {}'.format(i))
    for j in range(i,d-i):
      print(j)
      circuit.add_parametric_RY_gate(j,angle)
      print('Ry on {}'.format(j))
      if j % 2 == 0:
        circuit.add_CNOT_gate(j,j+1 % d)
        print('CN on {},{}'.format(j,j+1))
      # circuit.add_parametric_RZ_gate(j,angle)
      # print('Rz on {}'.format(j))

  f = int(np.round(d/2))
  print('final gate {}'.format(f))
  circuit.add_parametric_RX_gate(f,angle)
  print('Rx on {}'.format(f))

  #Hadamard gate for the ancila qubits
  for i in range(a):
    circuit.add_H_gate(d+i)
    print('had gate on {}'.format(d+i))
  
  # #Entangle ancila qubits
  # circuit.add_CNOT_gate(f+1,d)
  # print('CN on {} {}'.format(f+1,d))
  # circuit.add_CNOT_gate(f,d+1)
  # print('CN on {} {}'.format(f,d+1))
  


  # for i in range(a):
  #   circuit.add_parametric_RX_gate(d + i,angle)
  #   circuit.add_parametric_RY_gate(d + i,angle)
  #   circuit.add_parametric_RZ_gate(d + i,angle)
    
  #circuit.add_CNOT_gate(d,d+1)

  #Measure nth gate
  return circuit

In [ ]:
circuit = U_conv_6(8)

In [ ]:
#Simple n qubit amplitude embedding 
def pointwise_identity_embedding(point):
  n = len(point)
  state = QuantumState(int(np.log2(n)))
  point = point / np.linalg.norm(point)
  state.load(point)

  return state

#Qubit embedding
def pointwise_qubit_embedding(point):
  n = len(point)
  state = QuantumState(n)
  state.set_zero_state()
  for i in range(n):
    ry_gate = RY(i, point[i])
    ry_gate.update_quantum_state(state)

  return state

#Discrimination circuit
def U_conv_6(d):
  a = 2 #2 ancila qubits
  n = d + a

  #Metadata
  angle = 0
  # construct parametric quantum circuit with random rotation
  circuit = ParametricQuantumCircuit(n)

  # for i in range(n):
  #   circuit.add_H_gate(i)

  for i in range(math.ceil(d/2)+1):
    print('Pass {}'.format(i))
    for j in range(i,d-i):
      print(j)
      circuit.add_parametric_RY_gate(j,angle)
      if j % 2 == 0:
        circuit.add_CNOT_gate(j,j+1 % d)
      circuit.add_parametric_RZ_gate(j,angle)

  f = int(np.round(d/2))
  print('final gate {}'.format(f))
  circuit.add_parametric_RX_gate(f,angle)

  # #Hadamard gate for the ancila qubits
  # for i in range(a):
  #   circuit.add_H_gate(d+i)
  
  #Entangle ancila qubits
  circuit.add_CNOT_gate(f+1,d)
  circuit.add_CNOT_gate(f,d+1)


  for i in range(a):
    circuit.add_parametric_RX_gate(d + i,angle)
    circuit.add_parametric_RY_gate(d + i,angle)
    circuit.add_parametric_RZ_gate(d + i,angle)

  circuit.add_CNOT_gate(d,d+1)

  #Measure nth gate
  return circuit

#Discrimination circuit
def U_conv_6_double(n):
  #Metadata
  angle = 0
  # construct parametric quantum circuit with random rotation
  circuit = ParametricQuantumCircuit(n)
    
  for i in range(math.ceil(n/2)+1):
    print('Pass {}'.format(i))
    for j in range(i,n-i):
      print(j)
      circuit.add_parametric_RY_gate(j,angle)
    if j % 2 == 0:
      circuit.add_CNOT_gate(j,j+1 % n)

  for i in range(math.ceil(n/2)+1):
    print('Pass {}'.format(i+math.ceil(n/2)+1))
    for j in range(i,n-i):
      print(j+math.ceil(n/2)+1)
      circuit.add_parametric_RY_gate(j,angle)
    if j % 2 == 0:
      circuit.add_CNOT_gate(j,j+1 % n)

  f = int(np.round(n/2))
  print('final gate {}'.format(f))
  circuit.add_parametric_RY_gate(f,angle)

  return circuit

def interleave_anzats(n):

  #Metadata
  angle = 0
  # construct parametric quantum circuit with random rotation
  circuit = ParametricQuantumCircuit(n+1)

  # target_layer_1 = np.array([0,1,2,3], dtype = 'int')
  
  circuit.add_H_gate(n)
    
  for i in range(math.ceil(n/2)+1):
    print('Pass {}'.format(i))
    for j in range(i,n-i):
      print(j)
      circuit.add_parametric_RY_gate(j,angle)
    if j % 2 == 0:
      circuit.add_CNOT_gate(j,j+1 % n)

  for i in range(math.ceil(n/2)+1):
    print('Pass {}'.format(i))
    for j in range(i,n-i):
      print(j)
      circuit.add_parametric_RY_gate(j,angle)
    if j % 2 == 1:
      circuit.add_CNOT_gate(j,j+1 % n)

  
  circuit.add_CNOT_gate(n,n+1)
 

  return circuit

## Ising Model Classification

The goal now is to create a large dataset of Ising ground states and to classifiy them depending on their magnetisation properties. 

The key detail is the relative strength of the magnet field w.r.t the site to site interactions. Since we are interested in the symmetry breaking properties a constant field in the x axis with a low strength will be considered. The strength of the field sets the sclae at which we see the interactions occuring. 

### Create data

In [ ]:
def _get_data(n,datapoints,epsilon):
  Js = np.linspace(-10,10,datapoints)
  hs = np.linspace(0,10,datapoints)
  field_constants = np.array(list(it.product(Js,hs)))

  magnetisations = np.zeros(field_constants.shape[0])
  Ground_states = np.zeros((field_constants.shape[0],2**n),dtype = complex)  
  for i in range(len(field_constants)):
    H_dummy = qubit_operator_sparse(L_F_I(n,field_constants[i,0],field_constants[i,1],periodic_bc = True))
    
    m,g,ge = magnetisation_expectation_Z(n,H_dummy)
    magnetisations[i] = m[0,0]
    g = g.reshape(2**n)

    Ground_states[i,:] = g 
  magnetisations = magnetisations.reshape(datapoints**2)
  relative_field_constants = field_constants
  
  c = np.matrix(np.random.rand(2**n,2**n)+ 1j*np.random.rand(2**n,2**n))
  d = np.conjugate(c.T)
  h = c + d
  h = epsilon*h
  U = scipy.linalg.expm(1j*h)
  for i in range(Ground_states.shape[0]):
    Ground_states[i,:] = U @ Ground_states[i,:]
  return  np.abs(magnetisations),Ground_states,relative_field_constants
 

### Test train split

In [ ]:
import tensorflow as tf
from keras.callbacks import History 

In [ ]:
n = 12
datapoints = 60
Magnetisations_12 , groundstates_12 , field_constants_12 = _get_data(n,datapoints,0)


#Test, train set preparation
ising_data_12 = pd.DataFrame(groundstates_12)
ising_data_12 = pd.DataFrame(ising_data_12.iloc[:,:])
labels_12 = np.zeros(len(Magnetisations_12))

for i in range(len(Magnetisations_12)):
  if Magnetisations_12[i] <= 0.5:
    labels_12[i] = 0
  else:
    labels_12[i] = 1

ising_data_12.insert(2**n,'Js', field_constants_12[:,0])
ising_data_12.insert(2**n,'hs', field_constants_12[:,1])
ising_data_12.insert(2**n,'labels',labels_12)
ising_data_12.insert(2**n,'True Magnetisations', Magnetisations_12)

ising_data_12.to_csv('/content/drive/MyDrive/Data/ising_data_12.csv')

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(ising_data.iloc[:,0:2**n], ising_data.iloc[:,2**n:2**n+4], test_size=0.7)

In [ ]:
ising_data_12 = pd.read_csv('/content/drive/MyDrive/Data/ising_data_12.csv')

In [ ]:
n = 8
datapoints = 30
Magnetisations_8 , groundstates_8 , field_constants_8 = _get_data(n,datapoints,0)


#Test, train set preparation
ising_data_8 = pd.DataFrame(groundstates_8)
ising_data_8 = pd.DataFrame(ising_data_8.iloc[:,:])
labels_8 = np.zeros(len(Magnetisations_8))

for i in range(len(Magnetisations_8)):
  if Magnetisations_8[i] <= 0.5:
    labels_8[i] = 0
  else:
    labels_8[i] = 1

ising_data_8.insert(2**n,'Js', field_constants_8[:,0])
ising_data_8.insert(2**n,'hs', field_constants_8[:,1])
ising_data_8.insert(2**n,'labels',labels_8)
ising_data_8.insert(2**n,'True Magnetisations', Magnetisations_8)

ising_data_8.to_csv('/content/drive/MyDrive/Data/ising_data_8.csv')

In [ ]:
n=8
ising_data_8 = pd.read_csv('/content/drive/MyDrive/Data/ising_data_8.csv').iloc[:,1:]
ising_data_8.iloc[:,:2**n] = np.array(ising_data_8.iloc[:,:2**n],dtype = complex)

In [ ]:
ising_data_8

In [ ]:
n = 6
datapoints = 60
Magnetisations_6 , groundstates_6 , field_constants_6 = _get_data(n,datapoints,0)


#Test, train set preparation
ising_data_6 = pd.DataFrame(groundstates_6)
ising_data_6 = pd.DataFrame(ising_data_6.iloc[:,:])
labels_6 = np.zeros(len(Magnetisations_6))

for i in range(len(Magnetisations_6)):
  if Magnetisations_6[i] <= 0.5:
    labels_6[i] = 0
  else:
    labels_6[i] = 1

ising_data_6.insert(2**n,'Js', field_constants_6[:,0])
ising_data_6.insert(2**n,'hs', field_constants_6[:,1])
ising_data_6.insert(2**n,'labels',labels_6)
ising_data_6.insert(2**n,'True Magnetisations', Magnetisations_6)

ising_data_6.to_csv('/content/drive/MyDrive/Data/ising_data_6.csv')

In [ ]:
n = 6
ising_data_6 = pd.read_csv('/content/drive/MyDrive/Data/ising_data_6.csv').iloc[:,1:]
ising_data_6.iloc[:,:2**n] = np.array(ising_data_6.iloc[:,:2**n],dtype = complex)

In [ ]:
n = 4
datapoints = 100
Magnetisations_4 , groundstates_4 , field_constants_4 = _get_data(n,datapoints,0)

#Test, train set preparation
ising_data_4 = pd.DataFrame(groundstates_4)
ising_data_4 = pd.DataFrame(ising_data_4.iloc[:,:])
labels_4 = np.zeros(len(Magnetisations_4))

for i in range(len(Magnetisations_4)):
  if Magnetisations_4[i] <= 0.5:
    labels_4[i] = 0
  else:
    labels_4[i] = 1

ising_data_4.insert(2**n,'Js', field_constants_4[:,0])
ising_data_4.insert(2**n,'hs', field_constants_4[:,1])
ising_data_4.insert(2**n,'labels',labels_4)
ising_data_4.insert(2**n,'True Magnetisations', Magnetisations_4)

ising_data_4.to_csv('/content/drive/MyDrive/Data/ising_data_4.csv')

In [ ]:
n=4
ising_data_4 = pd.read_csv('/content/drive/MyDrive/Data/ising_data_4.csv').iloc[:,1:]
ising_data_4.iloc[:,:2**n] = np.array(ising_data_4.iloc[:,:2**n],dtype = complex)

In [ ]:
n = 8
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(ising_data_8.iloc[:,0:2**n], ising_data_8.iloc[:,2**n:2**n+4], test_size=0.7)
# n = 6
# X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(ising_data_6.iloc[:,0:2**n], ising_data_6.iloc[:,2**n:2**n+4], test_size=0.7)
# n = 4
# X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(ising_data_4.iloc[:,0:2**n], ising_data_4.iloc[:,2**n:2**n+4], test_size=0.7)


In [ ]:
ising_data_8

### Classification with tensorflow

In [ ]:
n = 8

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense((2^n), activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model_1.summary()

In [ ]:
model_1 = model
model_2 = model
model_3 = model
model_4 = model

In [ ]:
model_1.fit(X_train_1,y_train_1.iloc[:,1], epochs=100)

In [ ]:
model_2.fit(X_train_2,y_train_2.iloc[:,1], epochs=100)

In [ ]:
model_3.fit(X_train_3,y_train_3.iloc[:,1], epochs=100)

In [ ]:
model_4.fit(X_train_4,y_train_4.iloc[:,1], epochs=100)

In [ ]:
def find_nn_predictions(model,dataset):
  dummy = model.predict(dataset)
  model_predictions = np.zeros(model.predict(dataset).shape[0])
  for i in range(model_predictions.shape[0]):
    if dummy[i,0] > dummy[i,1]:
      model_predictions[i] = 0
    else:
      model_predictions[i] = 1
  return model_predictions

def get_accuracy(predicted_values,true_values):
  correct = 0
  for i in range(predicted_values.shape[0]):
    if predicted_values[i] == true_values[i]:
      correct +=1
  accuracy = (correct/len(predicted_values)) * 100
  return accuracy



predictions_1 = find_nn_predictions(model_1,X_test_1)
predictions_2 = find_nn_predictions(model_2,X_test_2)
predictions_3 = find_nn_predictions(model_3,X_test_3)
predictions_4 = find_nn_predictions(model_4,X_test_4)
  
get_accuracy(predictions_1,np.array(y_test_1.iloc[:,1]))

In [ ]:
get_accuracy(predictions_2,np.array(y_test_2.iloc[:,1]))

In [ ]:
get_accuracy(predictions_3,np.array(y_test_3.iloc[:,1]))

In [ ]:
get_accuracy(predictions_4,np.array(y_test_4.iloc[:,1]))

### Quantum Classifcations and comparison with longitudinal field

#### 12 qubits


In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = Axes3D(fig)
y = ising_data_12['Js']
x = ising_data_12['hs']
z = ising_data_12['True Magnetisations']

ax.plot_trisurf(x,y,z,  cmap=cm.jet, linewidth=0.2)
plt.title('Ising spin chain magnetisation')
ax.set_ylabel('Js')
ax.set_xlabel('hs')
ax.set_zlabel('Magnetisations')
plt.legend()

In [ ]:
n = 12
classifier_4 = MLP_2(n,U_conv_6,pointwise_identity_embedding)
classifier_4.train(X_train_4,y_train_1.iloc[:,1])
expectations_4 = classifier_1.predict(X_test_4.iloc[:,0:2**n])

def classify_bi(exp):
  if exp >= 0.5:
    pred = 1
  else:
    pred = 0
  return pred

 #Check the accuracy of the predictions 
correct_4 = 0

q_predictions_4 = np.zeros(len(X_test_4.iloc[:,0]))
for i in range(len(q_predictions_4)):
  q_predictions_4[i] = classify_bi(expectations_4[i])
  #print(q_predictions[i])
  if q_predictions_4[i] == y_test_4.iloc[i,1]:
    correct_4 += 1

accuracy_4 = (correct_4/len(q_predictions_4))*100

print('The binary classification for the Ising Model data set is ', accuracy_4 ,'% accurate ')

y_test_4.insert(3,'expectation values',expectations_4)
y_test_4.insert(4,'predicted classes',q_predictions_4)

y_test_4.sort_values('True Magnetisations')

#### 8 qubits

In [ ]:
z = ising_data_8['True Magnetisations']

In [ ]:
ising_data_8

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = Axes3D(fig)
y = ising_data_8['Js']
x = ising_data_8['hs']
z = ising_data_8['True Magnetisations']

ax.plot_trisurf(x,y,z,  cmap=cm.jet, linewidth=0.2)
plt.title('Ising spin chain magnetisation')
ax.set_ylabel('Js')
ax.set_xlabel('hs')
ax.set_zlabel('Magnetisations')
plt.savefig('8 qubit ISING magnetisation TRUE.pdf')
plt.legend()

In [ ]:
# plt.plot(np.linspace(-5,5,datapoints),ising_data['True Magnetisations'])
# #plt.xscale('log')
# plt.legend(['Magnetisation'])
# plt.title('Magnetisations')
# plt.xlabel('Relative transverse field strength')
# plt.ylabel('Magnetisation')

In [ ]:
n = 8
classifier_1 = MLP_2(n,U_conv_6,pointwise_identity_embedding)
classifier_1.train(X_train_1,y_train_1.iloc[:,1])
expectations_1 = classifier_1.predict(X_test_1.iloc[:,0:2**n])

def classify_bi(exp):
  if exp >= 0.5:
    pred = 1
  else:
    pred = 0
  return pred

 #Check the accuracy of the predictions 
correct_1 = 0

q_predictions_1 = np.zeros(len(X_test_1.iloc[:,0]))
for i in range(len(q_predictions_1)):
  q_predictions_1[i] = classify_bi(expectations_1[i])
  #print(q_predictions[i])
  if q_predictions_1[i] == y_test_1.iloc[i,1]:
    correct_1 += 1

accuracy_1 = (correct_1/len(q_predictions_1))*100

print('The binary classification for the Ising Model data set is ', accuracy_1 ,'% accurate ')

y_test_1.insert(3,'expectation values',expectations_1)
y_test_1.insert(4,'predicted classes',q_predictions_1)

y_test_1.sort_values('True Magnetisations')

In [ ]:
y_test_1

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = Axes3D(fig)
y = y_test_1['Js']
x = y_test_1['hs']
z = y_test_1['expectation values']

ax.plot_trisurf(x,y,z,  cmap=cm.jet, linewidth=0.2)
plt.title('Ising spin chain QNN expectation values')
ax.set_ylabel('Js')
ax.set_xlabel('hs')
ax.set_zlabel('Expectation Value')

plt.savefig('8 qubit ISING magnetisation PREDICTION.pdf')


In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = Axes3D(fig)
y = y_test_1['Js']
x = y_test_1['hs']
z = np.abs(y_test_1['expectation values'] - y_test_1['True Magnetisations'])

ax.plot_trisurf(x,y,z,  cmap=cm.jet, linewidth=0.2)
plt.title('Ising spin chain magnetism observable error')
ax.set_ylabel('Js')
ax.set_xlabel('hs')
ax.set_zlabel('Observable prediction error')
plt.savefig('8 qubit ISING magnetisation ERROR.pdf')
plt.legend()

In [ ]:
plt.plot(np.linspace(0,len(-y_test_1.iloc[:,0]),len(-y_test_1.iloc[:,0])),y_test_1.sort_values('True Magnetisations').iloc[:,0]) #magnetisation
plt.plot(np.linspace(0,len(-y_test_1.iloc[:,0]),len(-y_test_1.iloc[:,0])),y_test_1.sort_values('True Magnetisations').iloc[:,3]) #Expectations
# plt.plot(np.linspace(0,len(-y_test_1.iloc[:,0]),len(-y_test_1.iloc[:,0])),y_test_1.sort_values('True Magnetisations').iloc[:,1]) #Classes
# plt.plot(np.linspace(0,len(-y_test_1.iloc[:,0]),len(-y_test_1.iloc[:,0])),y_test_1.sort_values('True Magnetisations').iloc[:,4])#predicted classes
plt.title('Magnetisations and expectations of various field strengths')
plt.xlabel('Ground state')
# plt.legend(['magnetisation','expectation values','true labels','predicted classes'])
plt.legend(['magnetisation','expectation values'])

#### 6 qubits

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = Axes3D(fig)
y = ising_data_6['Js']
x = ising_data_6['hs']
z = ising_data_6['True Magnetisations']

ax.plot_trisurf(x,y,z,  cmap=cm.jet, linewidth=0.2)
plt.title('Ising spin chain magnetisation')
ax.set_ylabel('Js')
ax.set_xlabel('hs')
ax.set_zlabel('Magnetisations')
plt.legend()

In [ ]:
n = 6
classifier_2 = MLP_2(n,U_conv_6,pointwise_identity_embedding)
classifier_2.train(X_train_2,y_train_2.iloc[:,1])
expectations_2 = classifier_2.predict(X_test_2.iloc[:,0:2**n])

def classify_bi(exp):
  if exp >= 0.5:
    pred = 1
  else:
    pred = 0 
  return pred

 #Check the accuracy of the predictions 
correct_2 = 0

q_predictions_2 = np.zeros(len(X_test_2.iloc[:,0]))
for i in range(len(q_predictions_2)):
  q_predictions_2[i] = classify_bi(expectations_2[i])
  #print(q_predictions[i])
  if q_predictions_2[i] == y_test_2.iloc[i,1]:
    correct_2 += 1

accuracy_2 = (correct_2/len(q_predictions_2))*100

print('The binary classification for the Ising Model data set is ', accuracy_2 ,'% accurate ')

y_test_2.insert(3,'expectation values',expectations_2)
y_test_2.insert(4,'predicted classes',q_predictions_2)

y_test_2.sort_values('True Magnetisations')

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = Axes3D(fig)
y = y_test_2['Js']
x = y_test_2['hs']
z = y_test_2['expectation values']

ax.plot_trisurf(x,y,z,  cmap=cm.jet, linewidth=0.2)
plt.title('Ising spin chain QNN expectation values')
ax.set_ylabel('Js')
ax.set_xlabel('hs')
ax.set_zlabel('Expectation Value')
plt.legend()

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = Axes3D(fig)
y = y_test_2['Js']
x = y_test_2['hs']
z = np.abs(y_test_2['expectation values'] - y_test_2['True Magnetisations'])

ax.plot_trisurf(x,y,z,  cmap=cm.jet, linewidth=0.2)
plt.title('Ising spin chain magnetism observable error')
ax.set_ylabel('Js')
ax.set_xlabel('hs')
ax.set_zlabel('Observable prediction error')
plt.legend()

In [ ]:
plt.plot(np.linspace(0,len(-y_test_2.iloc[:,0]),len(-y_test_2.iloc[:,0])),y_test_2.sort_values('True Magnetisations').iloc[:,0]) #magnetisation
plt.plot(np.linspace(0,len(-y_test_2.iloc[:,0]),len(-y_test_2.iloc[:,0])),y_test_2.sort_values('True Magnetisations').iloc[:,3]) #Expectations
# plt.plot(np.linspace(0,len(-y_test_2.iloc[:,0]),len(-y_test_2.iloc[:,0])),y_test_2.sort_values('True Magnetisations').iloc[:,1]) #Classes
# plt.plot(np.linspace(0,len(-y_test_2.iloc[:,0]),len(-y_test_2.iloc[:,0])),y_test_2.sort_values('True Magnetisations').iloc[:,4])#predicted classes
plt.xlabel('Ground state')
plt.legend(['magnetisation','expectation values','true labels','predicted classes'])

#### 4 qubits

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = Axes3D(fig)
y = ising_data_4['Js']
x = ising_data_4['hs']
z = ising_data_4['True Magnetisations']

ax.plot_trisurf(x,y,z,  cmap=cm.jet, linewidth=0.2)
plt.title('Ising spin chain magnetisation')
ax.set_ylabel('Js')
ax.set_xlabel('hs')
ax.set_zlabel('Magnetisations')
plt.legend()

In [ ]:
n = 4
classifier_3 = MLP_2(n,U_conv_6,pointwise_identity_embedding)
classifier_3.train(X_train_3,y_train_3.iloc[:,1])
expectations_3 = classifier_3.predict(X_test_3.iloc[:,0:2**n])

def classify_bi(exp):
  if exp >= 0.5:
    pred = 1
  else:
    pred = 0
  return pred

 #Check the accuracy of the predictions 
correct_3 = 0

q_predictions_3 = np.zeros(len(X_test_3.iloc[:,0]))
for i in range(len(q_predictions_3)):
  q_predictions_3[i] = classify_bi(expectations_3[i])
  #print(q_predictions[i])
  if q_predictions_3[i] == y_test_3.iloc[i,1]:
    correct_3 += 1

accuracy_3 = (correct_3/len(q_predictions_3))*100

print('The binary classification for the Ising Model data set is ', accuracy_3 ,'% accurate ')

y_test_3.insert(3,'expectation values',expectations_3)
y_test_3.insert(4,'predicted classes',q_predictions_3)

y_test_3.sort_values('True Magnetisations')

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = Axes3D(fig)
y = y_test_3['Js']
x = y_test_3['hs']
z = y_test_3['expectation values']

ax.plot_trisurf(x,y,z,  cmap=cm.jet, linewidth=0.2)
plt.title('Ising spin chain QNN expectation values')
ax.set_ylabel('Js')
ax.set_xlabel('hs')
ax.set_zlabel('Expectation Value')
plt.legend()

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = Axes3D(fig)
y = y_test_3['Js']
x = y_test_3['hs']
z = np.abs(y_test_3['expectation values'] - y_test_3['True Magnetisations'])

ax.plot_trisurf(x,y,z,  cmap=cm.jet, linewidth=0.2)
plt.title('Ising spin chain magnetism observable error')
ax.set_ylabel('Js')
ax.set_xlabel('hs')
ax.set_zlabel('Observable prediction error')
plt.legend()

In [ ]:
plt.plot(np.linspace(0,len(-y_test_3.iloc[:,0]),len(-y_test_3.iloc[:,0])),y_test_3.sort_values('True Magnetisations').iloc[:,0]) #magnetisation
plt.plot(np.linspace(0,len(-y_test_3.iloc[:,0]),len(-y_test_3.iloc[:,0])),y_test_3.sort_values('True Magnetisations').iloc[:,3]) #Expectations
# plt.plot(np.linspace(0,len(-y_test_3.iloc[:,0]),len(-y_test_3.iloc[:,0])),y_test_3.sort_values('True Magnetisations').iloc[:,1]) #Classes
# plt.plot(np.linspace(0,len(-y_test_3.iloc[:,0]),len(-y_test_3.iloc[:,0])),y_test_3.sort_values('True Magnetisations').iloc[:,4])#predicted classes
plt.xlabel('Ground state')
plt.legend(['magnetisation','expectation values','true labels','predicted classes'])

### LIME

#### Quantum 

In [ ]:
plt.plot(np.linspace(0,len(-y_test_1.iloc[:,0]),len(-y_test_1.iloc[:,0])),y_test_1.sort_values('True Magnetisations').iloc[:,0]) #magnetisation
plt.plot(np.linspace(0,len(-y_test_1.iloc[:,0]),len(-y_test_1.iloc[:,0])),y_test_1.sort_values('True Magnetisations').iloc[:,3]) #Expectations
plt.plot(np.linspace(0,len(-y_test_1.iloc[:,0]),len(-y_test_1.iloc[:,0])),y_test_1.sort_values('True Magnetisations').iloc[:,1]) #Classes
plt.plot(np.linspace(0,len(-y_test_1.iloc[:,0]),len(-y_test_1.iloc[:,0])),y_test_1.sort_values('True Magnetisations').iloc[:,4])#predicted classes
plt.xlabel('Ground state')
plt.legend(['magnetisation','expectation values','true labels','predicted classes'])

In [ ]:
classifier = classifier_1

In [ ]:
field_features

In [ ]:
classifier.create_probabilities_Ising_field(field_features.iloc[:,:].values[42])

In [ ]:
np.array(X_test_1.iloc[42,0:2**4])

In [ ]:
classifier.predict(np.array(X_test_1.iloc[:,0:2**4]))[42]

In [ ]:
classifier.create_probabilities_Ising_field(np.array([field_features.iloc[:,:].values[43]]))

In [ ]:
classifier.create_probabilities_Ising_field(field_features.iloc[:,:].values[11])

In [ ]:
q_explainer = lime.lime_tabular.LimeTabularExplainer(field_features.iloc[:,:].values, feature_names=['J','h'], class_names=['Ferromagnetic','Anti-ferromagnetic'], discretize_continuous=True)
q_exp_1 = q_explainer.explain_instance(field_features.iloc[:,:].values[3], classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)
q_exp_1_1feature = q_explainer.explain_instance(field_features.iloc[:,:].values[3], classifier.create_probabilities_Ising_field, num_features=1, top_labels=1)


##### 4 qubits

In [ ]:
q_exp_1.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_exp_1_1feature.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_explainer = lime.lime_tabular.LimeTabularExplainer(field_features.iloc[:,:].values, feature_names=['J','h'], class_names=['Ferromagnetic','Anti-ferromagnetic'], discretize_continuous=True)
q_exp_2 = q_explainer.explain_instance(field_features.iloc[:,:].values[1], classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)
q_exp_2_1feature = q_explainer.explain_instance(field_features.iloc[:,:].values[1], classifier.create_probabilities_Ising_field, num_features=1, top_labels=1)


In [ ]:
q_exp_2.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_exp_2_1feature.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_explainer = lime.lime_tabular.LimeTabularExplainer(field_features.iloc[:,:].values, feature_names=['J','h'],class_names=['Ferromagnetic','Anti-ferromagnetic'], discretize_continuous=True)
q_exp_3 = q_explainer.explain_instance(field_features.iloc[:,:].values[2], classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)
q_exp_3_1feature = q_explainer.explain_instance(field_features.iloc[:,:].values[2], classifier.create_probabilities_Ising_field, num_features=1, top_labels=1)


In [ ]:
q_exp_3.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_exp_3_1feature.show_in_notebook(show_table=True, show_all=True)

##### 6 qubits

In [ ]:
q_explainer = lime.lime_tabular.LimeTabularExplainer(field_features.iloc[:,:].values, feature_names=['J','h'],class_names=['Ferromagnetic','Anti-ferromagnetic'], discretize_continuous=True)
q_exp_4 = q_explainer.explain_instance(field_features.iloc[:,:].values[87], classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)
q_exp_4_1feature = q_explainer.explain_instance(field_features.iloc[:,:].values[87], classifier.create_probabilities_Ising_field, num_features=1, top_labels=1)


In [ ]:
q_exp_4.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_exp_4_1feature.show_in_notebook(show_table=True, show_all=True)

##### 8 qubits

In [ ]:
y_test = y_test_1
field_features = pd.DataFrame({'Js' : y_test['Js'],'hs' : y_test['hs']})

In [ ]:
#Run me next
#trial_points = np.array([[-8,0.1],[-2.5,-1],[-5,7.5],[-0.9,-7.5],[1,-3],[1,0.5],[7,9]])

alpha = np.array([[-8,3.5],[-8,10],[1,0.5],[-0.9,-7.5],[1,-3],[1,0.5],[7,9]])

In [ ]:
#True magnetisations
n=8

for i in range(alpha.shape[0]):
  m,gs,ge = magnetisation_expectation_Z(n,qubit_operator_sparse(L_F_I(8,alpha[i,0],alpha[i,1],periodic_bc = True)))
  print(alpha[i,:],m.real,classifier.predict(np.array([gs])))

In [ ]:
classifier.predict(np.array([gs]))

In [ ]:
#Predicted magnetisations
for i in range(trial_points.shape[0]):
  print(trial_points[i,:] ,classifier_1.create_probabilities_Ising_field(trial_points[i,:]))

In [ ]:
l2n1

In [ ]:
l1n1

In [ ]:
# q_explainer = lime.lime_tabular.LimeTabularExplainer(field_features.iloc[:,:].values, feature_names=['J','h'],class_names=['Anti-ferromagnetic','Ferromagnetic'], discretize_continuous=True)
# classifier = classifier_1
# l2n1 = []
# l1n1 = []
for i in range(5,trial_points.shape[0]):
   l2n1.append(q_explainer.explain_instance(trial_points[i,:], classifier.create_probabilities_Ising_field, num_features=2, top_labels=1))
   l1n1.append(q_explainer.explain_instance(trial_points[i,:], classifier.create_probabilities_Ising_field, num_features=1, top_labels=1))

Top label only

In [ ]:
#1 feature explainations 
for i in range(trial_points.shape[0]):
  print(l1n1[i].show_in_notebook(show_table=True, show_all=True))

In [ ]:
#1 feature explainations 
for i in range(trial_points.shape[0]):
  print(l2n1[i].show_in_notebook(show_table=True, show_all=True))

In [ ]:
trial_points[1,:]

In [ ]:
q_exp_c = q_explainer.explain_instance(np.array([-1,0.5]), classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)
q_exp_d = q_explainer.explain_instance(np.array([1,0.5]), classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)


In [ ]:
#q_exp_e = q_explainer.explain_instance(np.array([-4,3.5]), classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)
x§q_exp_f = q_explainer.explain_instance(np.array([-1,1.25]), classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)


In [ ]:
q_exp_e.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_exp_f.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_exp_c.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_exp_d.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_explainer = lime.lime_tabular.LimeTabularExplainer(field_features.iloc[:,:].values, feature_names=['J','h'],class_names=['Anti-ferromagnetic','Ferromagnetic'], discretize_continuous=True)
q_exp_a = q_explainer.explain_instance(np.array([-1, 7 ]), classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)
q_exp_b = q_explainer.explain_instance(np.array([-1,0.5]), classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)
q_exp_e = q_explainer.explain_instance(np.array([-1, -7 ]), classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)


In [ ]:
q_exp_10 = q_explainer.explain_instance(np.array([-1, 10 ]), classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)


In [ ]:
classifier.create_probabilities_Ising_field(np.array([[-1, 10 ]]))

In [ ]:
q_exp_10.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_exp_a.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_exp_b.show_in_notebook(show_table=True, show_all=True)

In [ ]:
q_exp_e.show_in_notebook(show_table=True, show_all=True)

In [ ]:
bq_explainer.q_explainer(np.array[-1,1.23], classifier.create_probabilities_Ising_field, num_features=1, top_labels=1)

In [ ]:
l2n

In [ ]:
l1n

In [ ]:
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = Axes3D(fig)
y = ising_data_8['Js']
x = ising_data_8['hs']
z = ising_data_8['True Magnetisations']

ax.plot_trisurf(x,y,z,  cmap=cm.jet, linewidth=0.2)
plt.title('Ising spin chain magnetisation')
ax.set_ylabel('Js')
ax.set_xlabel('hs')
ax.set_zlabel('Magnetisations')
plt.legend()

Both Labels

In [ ]:
q_explainer = lime.lime_tabular.LimeTabularExplainer(field_features.iloc[:,:].values, feature_names=['J','h'],class_names=['Anti-ferromagnetic','Ferromagnetic'], discretize_continuous=True)
classifier = classifier_1
l2n2 = []
l1n2 = []
for i in range(0,trial_points.shape[0]):
   l2n2.append(q_explainer.explain_instance(trial_points[i,:], classifier.create_probabilities_Ising_field, num_features=2, top_labels=2))
   l1n2.append(q_explainer.explain_instance(trial_points[i,:], classifier.create_probabilities_Ising_field, num_features=1, top_labels=2))

In [ ]:
#1 feature explainations 
for i in range(trial_points.shape[0]):
  print(l1n2[i].show_in_notebook(show_table=True, show_all=True))

In [ ]:
#2 feature explainations 
for i in range(trial_points.shape[0]):
  print(l2n2[i].show_in_notebook(show_table=True, show_all=True))

#### ANN

In [ ]:
ann_explainer = lime.lime_tabular.LimeTabularExplainer(field_features.iloc[:,:].values, feature_names=['Site coupling strength','Transverse field strength'], class_names=['Ferromagnetic','Anti-ferromagnetic'], discretize_continuous=True)
ann_exp = q_explainer.explain_instance(field_features.iloc[:,:].values[2], classifier.create_probabilities_Ising_field, num_features=2, top_labels=1)


ann_exp.show_in_notebook(show_table=True, show_all=True)

In [ ]:
model_1(cnn.inputs, cnn.get_layer(theNameYouWant).outputs)

In [ ]:
Model(cnn.inputs, cnn.get_layer(theNameYouWant).outputs)

In [ ]:
 def ann_output_probabilities(model,input_X):

    if input_X.ndim == 1:
      input_X = np.reshape(input_X,(1,input_X.shape[0]))  

    # print('The input field strengths are: ',input_X)
    probabilities = np.zeros((input_X.shape[0],2))

    for i in range(input_X.shape[0]):
      E_vals,E_vecs = linalg.eigs(qubit_operator_sparse(T_F_I(n,J =input_X[i,0],h = input_X[i,1]))) #sparse Hamiltonian
      gs = E_vecs[:,np.argmin(E_vals)]
      # print(gs,gs.shape)
      ge = np.amin(E_vals)
      e = model_1(np.array([gs]), cnn.get_layer(theNameYouWant).outputs)
      e = model.predict(np.array([gs]))
      # print('the expectation value is: ', e)
      p_0 = 1-e
      p_1 = e
      probabilities[i,:] = np.reshape(np.array([p_0, p_1]),probabilities[i,:].shape)
      probabilities[i,:] = probabilities[i,:]/(probabilities[i,:].sum())           
      
    return probabilities

#### SK_learn

In [ ]:
y_test_1

In [ ]:
#Call model from sklearn
rfI = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rfI.fit(np.real(X_train_1), y_train_1.iloc[:,1])
rfI_classical_predictions = rfI.predict(np.real(X_test_1))

In [ ]:
classical_explainer = lime.lime_tabular.LimeTabularExplainer(np.real(X_test_1.values),  class_names=['Ferromagnetic','Paramagentic'], discretize_continuous=True)
classical_exp = classical_explainer.explain_instance(np.real(X_test_1.values[42]), rfI.predict_proba, num_features=2, top_labels=1)

classical_exp.show_in_notebook(show_table=True, show_all=True)

Need to apply the random forest classifeir to the quantum data

## Kitaev Square Octagon lattice

In [ ]:
def kitaev_square_octagon(J_x,J_y,J_z,h,n):
  lattice_8_qubits = (QubitOperator(f'Z0 Z4',-J_z) + 
                      QubitOperator(f'X0 X1',-J_x) + 
                      QubitOperator(f'Y0 Y2',-J_y) + 
                      QubitOperator(f'Z1 Z5',-J_z) + 
                      QubitOperator(f'Y1 Y3',-J_y) + 
                      QubitOperator(f'Z3 Z6',-J_z) + 
                      QubitOperator(f'X3 X2',-J_x) +
                      QubitOperator(f'Z2 Z7',-J_z))
  if n == 8:
    for i in range(n):
      lattice_8_qubits += QubitOperator('X{}'.format(i),h) + QubitOperator('Y{}'.format(i),h) + QubitOperator('Z{}'.format(i),h)
  

  
  lattice_16_qubits = (QubitOperator(f'Z0 Z4',-J_z) + 
                      QubitOperator(f'X0 X1',-J_x) + 
                      QubitOperator(f'Y0 Y2',-J_y) + 
                      QubitOperator(f'Z1 Z5',-J_z) + 
                      QubitOperator(f'Y1 Y3',-J_y) + 
                      QubitOperator(f'Z3 Z6',-J_z) + 
                      QubitOperator(f'X3 X2',-J_x) +
                      QubitOperator(f'Z2 Z7',-J_x) +
                      QubitOperator(f'X5 X12',-J_x) + 
                      QubitOperator(f'X6 X15',-J_x) +
                      QubitOperator(f'Z8 Z12',-J_z) + 
                      QubitOperator(f'X8 X9',-J_x) + 
                      QubitOperator(f'Y8 Y10',-J_y) + 
                      QubitOperator(f'Z9 Z13',-J_z) + 
                      QubitOperator(f'Y9 Y11',-J_y) + 
                      QubitOperator(f'Z11 Z14',-J_z) + 
                      QubitOperator(f'X11 X10',-J_x) +
                      QubitOperator(f'Z10 Z15',-J_z))
  
  if n == 16:
    for i in range(n):
      lattice_16_qubits += QubitOperator('X{}'.format(i),h) + QubitOperator('Y{}'.format(i),h) + QubitOperator('Z{}'.format(i),h)

  if n == 8:
    lattice =  lattice_8_qubits
  elif n == 16:
    lattice = lattice_16_qubits
  else:
    lattice =  'Please choose n = 8 or n = 16 only'
  return lattice

In [ ]:
E_vals,E_vecs = linalg.eigs(qubit_operator_sparse(kitaev_square_octagon(0.1,0.1,1,0,8)))
E_vecs.shape

### Data generation

In [ ]:
### TESTING CELL

# # Create data
# J_x = np.linspace(-5,5,datapoints)
# J_y = np.linspace(-5,5,datapoints)
# J_z = np.linspace(-5,5,1)
# h = np.linspace(-1,1,3)
# kitaev_parameters = list(it.product(J_x,J_y,J_z,h))

# # Results arrays
# magnetisations = np.zeros(len(kitaev_parameters))
# Ground_states = np.zeros((len(kitaev_parameters),2**n),dtype = complex)  
# energy = np.zeros(len(kitaev_parameters))

In [ ]:
def kitaev_octagon_get_data(n,datapoints,epsilon):
  #The cardinality actually datapoints^2 since we take all pair wise combinations

  # # Noise
  # c = np.matrix(np.random.rand(2**n,2**n)+ 1j*np.random.rand(2**n,2**n))
  # d = np.conjugate(c.T)
  # h = c + d
  # h = epsilon*h
  # U = scipy.linalg.expm(1j*h)

  # Create data
  J_x = np.linspace(-5,5,datapoints)
  J_y = np.linspace(-5,5,datapoints)
  J_z = np.linspace(-5,5,datapoints)
  h = np.linspace(-1,1,datapoints)
  kitaev_parameters = list(it.product(J_x,J_y,J_z,h))

  # Results arrays
  k_magnetisations = np.zeros(len(kitaev_parameters))
  Ground_states = np.zeros((len(kitaev_parameters),2**n),dtype = complex)  
  energy = np.zeros(len(kitaev_parameters))

  i = 0

  parameter = (J_x,J_y,J_z,h)

  for parameter in kitaev_parameters:
    parameter = kitaev_parameters[i]
    H_dummy = qubit_operator_sparse(kitaev_square_octagon(parameter[0],
                                                          parameter[1],
                                                          parameter[2],
                                                          parameter[3],
                                                          n))
    
    # # Apply noise transform
    # H_dummy = U @ H_dummy
                                                          
    m,g,e = magnetisation_expectation(n,H_dummy)
    k_magnetisations[i] = m[0,0]
    g = g.reshape(2**n)
    Ground_states[i,:] = g 
    e = np.array(e).real
    energy[i] = e
    i+= 1
                                                          
  return kitaev_parameters,magnetisations,Ground_states,energy

In [ ]:
#Create and format testing data
datapoints = 7
n=8
kitaev_parameters,magnetisations,Ground_states,energy = kitaev_octagon_get_data(8,datapoints,0)

kitaev_ground_states = pd.DataFrame(Ground_states)
kitaev_data = pd.DataFrame([kitaev_parameters,magnetisations,np.round(Ground_states,3),energy]).T
kitaev_data.columns = ['kitaev_parameters','magnetisations','Ground_states','energy']

#Transform data into useful formats

jx = np.zeros(kitaev_data.shape[0])
jy = np.zeros(kitaev_data.shape[0])
jz = np.zeros(kitaev_data.shape[0])
h = np.zeros(kitaev_data.shape[0])

for i in range(kitaev_data.shape[0]):
  jx[i] = (kitaev_data['kitaev_parameters'][i])[0]
  jy[i] = (kitaev_data['kitaev_parameters'][i])[1]
  jz[i] = (kitaev_data['kitaev_parameters'][i])[2]
  h[i] = (kitaev_data['kitaev_parameters'][i])[3]

parameters = pd.DataFrame([jx,jy,jz,h]).T
parameters.columns = ['jx','jy','jz','h']

kitaev_data = kitaev_data.iloc[:,1:]
kitaev_data.insert(0,'h',h)
kitaev_data.insert(0,'jz',jz)
kitaev_data.insert(0,'jy',jx)
kitaev_data.insert(0,'jx',jy)

ground_states = np.zeros((datapoints**4 ,2**8),dtype = complex)
for i in range(datapoints**4):
  ground_states[i,:] = kitaev_data['Ground_states'][i]

ground_states
kitaev_data = pd.concat([kitaev_data,pd.DataFrame(ground_states)],axis = 1)
kitaev_data = kitaev_data.drop(columns = 'Ground_states')


In [ ]:
kitaev_data.to_csv('/content/drive/MyDrive/Data/kitaev_8.csv')

In [ ]:
kitaev_data

### Import data

In [ ]:
imported_kitaev_data = pd.read_csv('/content/drive/MyDrive/Data/kitaev_8.csv')
kitaev_data = imported_kitaev_data.iloc[:,0:] # redundant remove index, only run once!

In [ ]:
kitaev_data = imported_kitaev_data.iloc[:,1:]

In [ ]:
kitaev_data.iloc[:,6:] = np.array(kitaev_data.iloc[:,6:],dtype = complex)

### Plot magnetisation at ground state energy

In [ ]:
np.max(kitaev_data['magnetisations'])

In [ ]:
### Reduced data set, setting h = 1
kitaev_data_reduced = kitaev_data
kitaev_data_reduced = kitaev_data_reduced[kitaev_data['jx'] == kitaev_data['jy']] 
kitaev_data_reduced = kitaev_data_reduced[kitaev_data['h'] == 1] 
kitaev_data_reduced.drop('jy',axis = 1)
kitaev_data_reduced.drop('h',axis = 1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D



ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.plot_trisurf(x, y, z, triangles=tri.triangles, cmap=plt.cm.Spectral)



In [ ]:
h_over_jz = np.zeros(kitaev_data_reduced.shape[0])
jx_over_jz = np.zeros(kitaev_data_reduced.shape[0])
for i in range(kitaev_data_reduced.shape[0]):
  h_over_jz[i] =  kitaev_data.iloc[i,3]/kitaev_data_reduced.iloc[i,2]
  jx_over_jz[i] = kitaev_data.iloc[i,0]/kitaev_data_reduced.iloc[i,2]
kitaev_data_reduced.insert(4,'h over jz',h_over_jz)
kitaev_data_reduced.insert(4,'jx over jz',jx_over_jz)
kitaev_data_reduced = kitaev_data_reduced.drop(['jx','jy','h','jz'],axis = 1).dropna()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

sns.set(style = "darkgrid")

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

x = np.array(kitaev_data_reduced['jx'])
y = np.array(kitaev_data_reduced['jz'])
z = np.array(kitaev_data_reduced['magnetisations'])
 

ax.set_xlabel("jx")
ax.set_ylabel("jz")
ax.set_zlabel("magnetisations")

ax.scatter(x, y, z)

plt.show()

In [ ]:
# Customize the z axis.
#ax.set_zlim(-1.01, 1.01)
# Plot the surface.
fig = plt.figure()
ax = fig.gca(projection='3d')
x2,y2 = np.meshgrid(x2,y2)
surf = ax.plot_trisurf(np.array(kitaev_data_reduced['jx']), 
                       np.array(kitaev_data_reduced['jz']), 
                       np.array(kitaev_data_reduced['magnetisations']), cmap=cm.plasma,
                       linewidth=0, antialiased=True)

#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
kitaev_data_reduced['magnetisations']

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x, y, z, c='y', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

### Plot ground state energy

In [ ]:
# Customize the z axis.
#ax.set_zlim(-1.01, 1.01)
# Plot the surface.
fig = plt.figure()
ax = fig.gca(projection='3d')

surf = ax.plot_trisurf(field_constants[:,1], field_constants[:,0], (energy), cmap=cm.plasma,
                       linewidth=0, antialiased=True)

#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
# Customize the z axis.
#ax.set_zlim(-1.01, 1.01)
# Plot the surface.
fig = plt.figure()
ax = fig.gca(projection='3d')

surf = ax.plot_trisurf(field_constants[:,0], field_constants[:,1], (energy), cmap=cm.plasma,
                       linewidth=0, antialiased=True)

#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
kitaev_data.to_csv('/content/kitaev')

### Kitaev phase diagram for noisey data

In [ ]:
noisy_field_magnetisations,noisy_field_ground_states,noisy_field_constants,noisy_enegy = kitaev_octagon_get_data(8,24,10)

In [ ]:
noisy_field_kitaev_data = pd.DataFrame([noisy_field_constants[:,0],noisy_field_constants[:,1],noisy_field_magnetisations,noisy_enegy]).T
noisy_field_kitaev_data.columns = ['zero_field_h_over_Jz','zero_field_Jxy_over_Jz','zero_field_magnetisations','noisy enegy']

In [ ]:
# Customize the z axis.
#ax.set_zlim(-1.01, 1.01)
# Plot the surface.
fig = plt.figure()
ax = fig.gca(projection='3d')

surf = ax.plot_trisurf(noisy_field_constants[:,1], noisy_field_constants[:,0], noisy_field_magnetisations, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

### Kmeans

In [ ]:
from sklearn.cluster import KMeans

X = np.array(kitaev_data.dropna())
kmeans = KMeans(n_clusters=4, random_state=42).fit(X)
kmeans.labels_
kitaev_data = kitaev_data.dropna()
kitaev_data.insert(3,'Kmeans_Labels',kmeans.labels_)

In [ ]:
kitaev_data.head(100)

In [ ]:
sns.pairplot(kitaev_data[['h_over_Jz','Jxy_over_Jz','Kmeans_Labels']], hue="Kmeans_Labels", palette = 'deep')

### Test train splits

Generate labels function:

In [ ]:
labels = np.zeros(kitaev_data.shape[0])

In [ ]:
def assign_phase_label(J_x,J_y,J_z,h):

  if J_z**2 < J_x**2 + J_y**2:
    label = 0
    
  elif J_z**2 > J_x**2 + J_y**2:
    label = 1

  return label

In [ ]:
def assign_phase_label(J_x,J_y,J_z,h):
  label = 0
  if J_z**2 < J_x**2 + J_y**2:
    label = 0
  elif J_z**2 > J_x**2 + J_y**2:
    label = 1

  return label

# Label data
count_0 = 0
count_1 = 0
labels = np.zeros(kitaev_data.shape[0])
for i in range(kitaev_data.shape[0]):
  labels[i] = assign_phase_label(kitaev_data['jx'][i],
                                 kitaev_data['jy'][i],
                                 kitaev_data['jz'][i],
                                 kitaev_data['h'][i])
  if labels[i] == 0:
    count_0 += 1
  else:
    count_1 += 1
print(count_0,count_1) 

kitaev_data.insert(4,'labels',labels)

In [ ]:
kitaev_data[kitaev_data['labels'] == 0]

In [ ]:
kitaev_data[kitaev_data['labels'] == 1]

In [ ]:
#Test, train set preparation
n = 8
K_X_train_1, K_X_test_1, K_y_train_1, K_y_test_1 = train_test_split(kitaev_data.iloc[:,7:2**n+8], kitaev_data['magnetisations'], test_size=0.2, random_state=42)
K_X_train_2, K_X_test_2, K_y_train_2, K_y_test_2 = train_test_split(kitaev_data.iloc[:,7:], kitaev_data['labels'], test_size=0.95, random_state=42)
K_X_train_3, K_X_letest_3, K_y_train_3, K_y_test_3 = train_test_split(kitaev_data.iloc[:,7:], kitaev_data['labels'], test_size=0.92, random_state=42)
K_X_train_4, K_X_test_4, K_y_train_4, K_y_test_4 = train_test_split(kitaev_data.iloc[:,7:], kitaev_data['labels'], test_size=0.9, random_state=42)

### Quantum classification

In [ ]:
n = 8
K_classifier_1 = MLP_2(n,U_conv_6,pointwise_identity_embedding)
K_classifier_1.train(K_X_train_1,K_y_train_1)
K_expectations_1 = K_classifier_1.predict(K_X_test_1)

def classify_bi(exp):
  if exp >= 0.5:
    pred = 1
  else:
    pred = 0
  return int(pred)

 #Check the accuracy of the predictions 
K_correct_1 = 0

K_q_predictions_1 = np.zeros(K_y_test_1.shape[0],dtype = 'int32')
for i in range(len(K_q_predictions_1)):
  K_q_predictions_1[i] = classify_bi(K_expectations_1[i])
  #print(q_predictions[i])
  if K_q_predictions_1[i] == np.array(K_y_test_1)[i]:
    K_correct_1 += 1

K_accuracy_1 = (K_correct_1/len(K_q_predictions_1))*100

print('The binary classification for the Kitaev chain data set is ', K_accuracy_1 ,'% accurate ')

K_y_test_1 = pd.DataFrame(K_y_test_1)


K_y_test_1.insert(1,'expectation values',K_expectations_1)
K_y_test_1.insert(2,'predicted toric phase classes',K_q_predictions_1)


In [ ]:
K_y_test_1

In [ ]:
sns.pairplot(K_y_test_1, hue="labels", palette = 'deep')

In [ ]:
kitaev_data.insert()

### Tensorflow

In [ ]:
K_model = tf.keras.Sequential([
    tf.keras.layers.Dense((2**8), activation='relu'),
    tf.keras.layers.Dense(17, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])

K_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
K_model_1 = K_model
K_model_2 = K_model
K_model_3 = K_model
K_model_4 = K_model

In [ ]:
K_model_1.fit(K_X_train_1,K_y_train_1, epochs=100)

In [ ]:
K_predictions_1 = find_nn_predictions(K_model_1,K_X_test_1)
  
get_accuracy(K_predictions_1,np.array(K_y_test_1))

## More complicated lattice and hamiltonians

### pyqula (Quantum lattice tight binding models) hamiltonian experimentation 
https://github.com/joselado/pyqula

In [ ]:
pip install pyqula

In [ ]:
## Lets make a pretty lattice pip install pyqula

In [ ]:
from pyqula import geometry
from pyqula import films
g = geometry.diamond_lattice()
g = films.geometry_film(g,nz=5)
h = g.get_hamiltonian()
(k,e) = h.get_bands()

### Pybinding package https://docs.pybinding.site/en/stable/tutorial/solvers.html

In [ ]:
pip install pybinding

In [ ]:
import pybinding as pb
pb.pltutils.use_style()
#run and compare to matplot lib default

In [ ]:
d = 0.2  # [nm] unit cell length
t = 1    # [eV] hopping energy

# create a simple 2D lattice with vectors a1 and a2
lattice = pb.Lattice(a1=[d, 0], a2=[0, d])
lattice.add_sublattices(
    ('A', [0, 0])  # add an atom called 'A' at position [0, 0]
)
lattice.add_hoppings(
    # (relative_index, from_sublattice, to_sublattice, energy)
    ([0, 1], 'A', 'A', t),
    ([1, 0], 'A', 'A', t)
)


In [ ]:
def square_lattice(d, t):
    lat = pb.Lattice(a1=[d, 0], a2=[0, d])
    lat.add_sublattices(('A', [0, 0]))
    lat.add_hoppings(([0, 1], 'A', 'A', t),
                     ([1, 0], 'A', 'A', t))
    return lat

# we can quickly set a shorter unit length `d`
lattice = square_lattice(d=0.1, t=1)
lattice.plot()
plt.show()

In [ ]:

def monolayer_graphene():
    a = 0.24595   # [nm] unit cell length
    a_cc = 0.142  # [nm] carbon-carbon distance
    t = -2.8      # [eV] nearest neighbour hopping

    lat = pb.Lattice(a1=[a, 0],
                     a2=[a/2, a/2 * sqrt(3)])
    lat.add_sublattices(('A', [0, -a_cc/2]),
                        ('B', [0,  a_cc/2]))
    lat.add_hoppings(
        # inside the main cell
        ([0,  0], 'A', 'B', t),
        # between neighboring cells
        ([1, -1], 'A', 'B', t),
        ([0, -1], 'A', 'B', t)
    )
    return lat

lattice = monolayer_graphene()
lattice.plot()
plt.show()

In [ ]:
model = pb.Model(monolayer_graphene(), pb.translational_symmetry())
model.hamiltonian.todense()

In [ ]:
graphine_model = pb.Model(lattice)
graphine_model.hamiltonian.todense()

In [ ]:
lattice = monolayer_graphene()
lattice.plot_brillouin_zone()

In [ ]:
from pybinding.repository import graphene

model = pb.Model(graphene.monolayer())
model.plot()

model = pb.Model(
    graphene.monolayer(),
    pb.primitive(a1=5, a2=3)
)
model.plot()
model.lattice.plot_vectors(position=[0.6, -0.25])

In [ ]:
model.hamiltonian.todense().shape

In [ ]:
"""Model a graphene ring structure and calculate the local density of states"""

pb.pltutils.use_style()


def ring(inner_radius, outer_radius):
    """A simple ring shape"""
    def contains(x, y, z):
        r = np.sqrt(x**2 + y**2)
        return np.logical_and(inner_radius < r, r < outer_radius)

    return pb.FreeformShape(contains, width=[2 * outer_radius, 2 * outer_radius])

# Simple shapes
rectangle = pb.rectangle(x=6, y=1)
hexagon = pb.regular_polygon(num_sides=6, radius=1.92, angle=np.pi/6)
circle = pb.circle(radius=0.6)

# Compose them naturally
shape = rectangle + hexagon - circle


model1 = pb.Model(
    graphene.monolayer(),
    shape  # length in nanometers
)

model.plot()

In [ ]:
# only solve for the 20 lowest energy eigenvalues
solver = pb.solver.arpack(model1, k=20)
ldos = solver.calc_spatial_ldos(energy=0, broadening=0.05)  # LDOS around 0 eV
ldos.plot(site_radius=(0.03, 0.12))
pb.pltutils.colorbar(label="LDOS")
plt.show()

In [ ]:
def circle(radius):
    def contains(x, y, z):
        return np.sqrt(x**2 + y**2) < radius
    return pb.FreeformShape(contains, width=[2*radius, 2*radius])

model2 = pb.Model(
    graphene.monolayer(),
    circle(radius=2.5)
)
model2.plot()

In [ ]:
# only solve for the 20 lowest energy eigenvalues
solver = pb.solver.arpack(model2, k=20)
ldos = solver.calc_spatial_ldos(energy=1, broadening=0.05)  # LDOS around 0 eV
ldos.plot(site_radius=(0.03, 0.12))
pb.pltutils.colorbar(label="LDOS")
plt.show()

In [ ]:
model.hamiltonian.todense()

### Quimb, a package for quantum many particle states and tensor networks https://quimb.readthedocs.io/en/latest/index_core.html

In [ ]:
pip install quimb

### Kwant to openFermion

https://github.com/alicjadut/kwant_to_openfermion
https://kwant-project.org/

#### Code to convert Kwant to Openfermion qubit operator

In [ ]:
##########Spin_lattice.py


import numpy as np
import openfermion
import kwant


# Define Pauli matrices
sigma_0 = np.array([[1., 0.], [0., 1.]])
sigma_x = np.array([[0., 1.], [1., 0.]])
sigma_y = np.array([[0., -1j], [1j, 0.]])
sigma_z = np.array([[1., 0.], [0., -1.]])
pauli_matrices = [sigma_0, sigma_x, sigma_y, sigma_z]
pauli_names = ['1', 'X', 'Y', 'Z']
#Define tensor products of pauli matrices
pauli_matrices4 = [np.kron(matrix1, matrix2)
                   for matrix1 in pauli_matrices
                   for matrix2 in pauli_matrices]
pauli_names4 = [(name1, name2) for name1 in pauli_names for name2 in pauli_names]


# Transformation from the computational basis to the Pauli basis
## 2x2 matrices
transformation_matrix2 = np.transpose(np.linalg.inv([m.reshape(4) for m in pauli_matrices]))
def _to_pauli_basis_2(m):
    vec = m.reshape(4)
    return np.dot(transformation_matrix2, vec)    
## 4x4 matrices
transformation_matrix4 = np.transpose(np.linalg.inv([m.reshape(4*4) for m in pauli_matrices4]))
def _to_pauli_basis_4(m):
    vec = m.reshape(4*4)
    return np.dot(transformation_matrix4, vec)

def to_pauli_basis(m):
    '''
    Return the decomposition of a matrix in the Pauli basis.
    Parameters
    ----------
    m: 2x2 or 4x4 array
        The matrix to be decomposed.
    Returns
    ----------
    v: vector of length 4 (for a 2x2 array) or 16 (for a 4x4 array)
        The coefficients in the Pauli basis, ordered as pauli_matrices or pauli_matrices4.
    '''
    try:
        return _to_pauli_basis_2(m)
    except:
        try:
            return _to_pauli_basis_4(m)
        except:
            raise ValueError('Cannot get Pauli coefficients.')



def _single_term_to_QubitOperator(val, ix1, ix2):
    '''
    Export single term of the hamiltonian to openfermion.
    Single term means either an onsite/single spin term at lattice site ix1 = ix2
    - such as Zeeman hamiltonian - or an interaction between two spins at sites ix1 and ix2.
    Parameters
    ----------
    val: 2x2 array for ix1 = ix2 or 4x4 array for ix1 =/= ix2
    ix1, ix2: int
        lattice position indices.
    Returns
    ----------
    op: openfermion.QubitOperator
    '''

    try:
        dims = val.shape
    except:
        raise ValueError(f'Expected a matrix, got {type(val)}.')

    pauli_coefs = to_pauli_basis(val)

    op = openfermion.QubitOperator()

    #On site terms
    if ix1 == ix2:
        assert dims == (2, 2), f'Onsite terms must be 2x2 matrices, got {dims}'
        for name, coef in zip(pauli_names, pauli_coefs):
            if name == '1':
                op += openfermion.QubitOperator('', coef)
            else:
                op += openfermion.QubitOperator(name+str(ix1), coef)
        return op

    #Interaction terms
    assert dims == (4, 4), f'Onsite terms must be 4x4 matrices, got {dims}'
    for name, coef in zip(pauli_names4, pauli_coefs):
        if name == ('1', '1'):
            op += openfermion.QubitOperator('', coef)
        elif name[0] == '1':
            op += openfermion.QubitOperator(name[1]+str(ix2), coef)
        elif name[1] == '1':
            op += openfermion.QubitOperator(name[0]+str(ix1), coef)
        else:
            op += openfermion.QubitOperator(name[0]+str(ix1)+' '+name[1]+str(ix2), coef)
    return op



def system_to_QubitOperator(sys):
    '''
    Export a spin lattice hamiltonian built in kwant to openfermion.
    Parameters
    ----------
    sys: kwant.system.FiniteSystem or kwant.system.InfiniteSystem
    Returns
    ----------
    ham: openfermion.QubitOperator
        The hamiltonian of sys as an openfermion object.
    '''

    if not isinstance(sys, kwant.system.System):
        raise TypeError(f'Expecting an instance of System, got {type(sys)}.')

    ham = openfermion.QubitOperator()

    #on site terms
    for lat_ix in sys.id_by_site.values():
        val = sys.hamiltonian(lat_ix, lat_ix)
        ham += _single_term_to_QubitOperator(val, lat_ix, lat_ix)

    #interaction terms
    for edge in range(sys.graph.num_edges):
        lat_ix1 = sys.graph.head(edge)
        lat_ix2 = sys.graph.tail(edge)
        val = sys.hamiltonian(lat_ix1, lat_ix2)/2
        ham += _single_term_to_QubitOperator(val, lat_ix1, lat_ix2)

    return ham

https://github.com/alicjadut/kwant_to_openfermion/blob/main/demo_notebooks/demo.ipynb

In [ ]:
##########Fermionic_system.py

import openfermion
import kwant

class Indexer:
    '''
    An object that matches lattice and spin indices to
    integer indices to be used by openfermion operators.
    '''

    def __init__(self):
        self._indexed_elements = []
        self._index_by_element = {}
        self._current_index = 0

    def index(self, el):
        '''
        Return an integer index for el.
        '''

        #If the element was not indexed already, create a new index
        if not el in self._indexed_elements:
            self._indexed_elements.append(el)
            self._index_by_element[el] = self._current_index
            self._current_index += 1

        return self._index_by_element[el]

    def element(self, ix):
        '''
        Return the element indexed by ix.
        '''
        return self._indexed_elements[ix]


def _single_term_to_FermionOperator(val, lat_ix1, lat_ix2, ind):
    '''
    Export single term of the hamiltonian to openfermion.
    Parameters
    ----------
    val: number or 2D array
    Returns
    ----------
    op: openfermion.FermionOperator
    '''
    try:
        n_spin1 = val.shape[0]
        n_spin2 = val.shape[1]

        op = openfermion.FermionOperator()
        for spin_ix1 in range(n_spin1):
            for spin_ix2 in range(n_spin2):
                ix1 = ind.index((lat_ix1, spin_ix1, n_spin1))
                ix2 = ind.index((lat_ix2, spin_ix2, n_spin2))
                op += openfermion.FermionOperator(f'{ix1}^ {ix2}', val[spin_ix1, spin_ix2])

        return op

    except:
        try:
            ix1 = ind.index((lat_ix1, 0, 1))
            ix2 = ind.index((lat_ix2, 0, 1))
            return openfermion.FermionOperator(f'{ix1}^ {ix2}', val)
        except:
            raise ValueError(f'''
            Cannot construct fermionic operator with
            indices {lat_ix1}, {lat_ix2}, value {val}''')

def system_to_FermionOperator(sys, return_indexer = False):
    '''
    Export the hamiltonian of a kwant system to openfermion.
    Parameters
    ----------
    sys: kwant.system.FiniteSystem or kwant.system.InfiniteSystem
    return_indexer: bool
    Returns
    ----------
    ham: openfermion.FermionOperator
        The hamiltonian of sys as an openfermion object.
    indexer: kwant_to_openfermion.Indexer
        An object that matched the interger indices used by the
        openfermion operator to site and spin indices.
    '''

    if not isinstance(sys, kwant.system.System):
        raise TypeError(f'Expecting an instance of System, got {type(sys)}')

    ham = openfermion.FermionOperator()
    ind = Indexer()

    #on site terms
    for lat_ix in sys.id_by_site.values():
        val = sys.hamiltonian(lat_ix, lat_ix)
        ham += _single_term_to_FermionOperator(val, lat_ix, lat_ix, ind)

    #hopping terms
    for edge in range(sys.graph.num_edges):
        lat_ix1 = sys.graph.head(edge)
        lat_ix2 = sys.graph.tail(edge)
        val = sys.hamiltonian(lat_ix1, lat_ix2)
        ham += _single_term_to_FermionOperator(val, lat_ix1, lat_ix2, ind)

    if return_indexer:
        return ham, ind
    return ham

### Kwant practice

In [ ]:
sys = kwant.Builder()

In [ ]:
#Sqaure lattice, momentum only hamiltonian

a = 1 # Lattice constant
lat = kwant.lattice.square(a)

In [ ]:
t = 1.0
W = 10
L = 30

# Define the scattering region

for i in range(L):
    for j in range(W):
        # On-site Hamiltonian
        sys[lat(i, j)] = 4 * t

        # Hopping in y-direction
        if j > 0:
            sys[lat(i, j), lat(i, j - 1)] = -t

        # Hopping in x-direction
        if i > 0:
            sys[lat(i, j), lat(i - 1, j)] = -t


In [ ]:
sym_left_lead = kwant.TranslationalSymmetry((-a, 0))
left_lead = kwant.Builder(sym_left_lead)

In [ ]:
for j in range(W):
    left_lead[lat(0, j)] = 4 * t
    if j > 0:
        left_lead[lat(0, j), lat(0, j - 1)] = -t
    left_lead[lat(1, j), lat(0, j)] = -t

In [ ]:
sys.attach_lead(left_lead)

In [ ]:
sym_right_lead = kwant.TranslationalSymmetry((a, 0))
right_lead = kwant.Builder(sym_right_lead)

for j in range(W):
    right_lead[lat(0, j)] = 4 * t
    if j > 0:
        right_lead[lat(0, j), lat(0, j - 1)] = -t
    right_lead[lat(1, j), lat(0, j)] = -t

sys.attach_lead(right_lead)

In [ ]:
kwant.plot(sys)

In [ ]:
sys = sys.finalized()

In [ ]:
system_to_QubitOperator(sys)

In [ ]:
import kwant

# For plotting
from matplotlib import pyplot


def make_system(a=1, t=1.0, W=10, L=30):
    # Start with an empty tight-binding system and a single square lattice.
    # `a` is the lattice constant (by default set to 1 for simplicity.
    lat = kwant.lattice.square(a)

    sys = kwant.Builder()

    sys[(lat(x, y) for x in range(L) for y in range(W))] = 4 * t

    sys[lat.neighbors()] = -t


    # lead = kwant.Builder(kwant.TranslationalSymmetry((-a, 0)))
    # lead[(lat(0, j) for j in range(W))] = 4 * t
    # lead[lat.neighbors()] = -t

    # sys.attach_lead(lead)
    # sys.attach_lead(lead.reversed())

    return sys

In [ ]:
sys = make_system(a=1, t=1.0, W=10, L=30)

In [ ]:
sys = sys.finalized()

In [ ]:
kwant.plot(sys)

In [ ]:
system_to_QubitOperator(sys)

### Update fuctions

In [ ]:
n = 13

In [ ]:
def sparse_magnetisation_expectation(n,H):
  ## Find the ground state of the hamiltonian 
  E_vals,E_vecs = linalg.eigsh(H) #Sparse Hamiltonian
  gs = E_vecs[:,np.argmin(E_vals)]

  ## Define the magnetisation
  M = QubitOperator()
  for i in range(n):
    M += QubitOperator(f"Z{i}", 1)
    M += QubitOperator(f"X{i}", 0.1)
  M = qubit_operator_sparse(M).real

  ## Compute the ground state magnetisation
  #print(gs.shape,M.shape)
  m_0 = (1/n) * np.conjugate(gs.T) @ M @ gs
  m_0 = m_0.real
  m_0 = m_0
  return m_0,gs

In [ ]:
gs.shape

In [ ]:
## Honeycombe ISING


In [ ]:
# Large system
# Or we can define a region (such as a circle 
# ) and use all sites inside this region.

r = 1.2
J = 1


def circle(pos):
        x, y = pos
        return x ** 2 + y ** 2 < r ** 2

lat = kwant.lattice.honeycomb()
syst = kwant.Builder()

syst[lat.shape(circle, (0,0))] = 1*sigma_0
    
for hopping, pauli_matrix in zip(lat.neighbors(), [sigma_x, sigma_y, sigma_z]):
    syst[hopping] = -J * np.kron(pauli_matrix, pauli_matrix)
    
syst = syst.finalized()
kwant.plot(syst)

ham = system_to_QubitOperator(syst)

In [ ]:
ham

In [ ]:
#Transverse field Ising Hamiltonian: Modifies a lattice hamiltionian with hopping terms 
#                                    by adding a magentic field interaction

def Honeycomb_Ising(H,n,h=1,periodic_bc = False):
  #H = QubitOperator()
  for i in range(n):
      H += QubitOperator(f"X{i}", -h)
      H += QubitOperator(f"Z{i}", 0.01)
  return H

In [ ]:
ham = Honeycomb_Ising(ham,n,h=1,periodic_bc = False)

ham

In [ ]:
H_sparse = qubit_operator_sparse(Honeycomb_Ising(ham,n,h = 2,periodic_bc = False))

In [ ]:
H_sparse.todense().real

In [ ]:
m,g = sparse_magnetisation_expectation(n, qubit_operator_sparse(Honeycomb_Ising(ham,n,h = 2,periodic_bc = False)))

In [ ]:
m

In [ ]:
def _get_data_honeycomb(H,n,datapoints):

  field_constants = np.linspace(-20,30,datapoints)
  magnetisations = np.zeros(len(field_constants))
  Ground_states = np.zeros((len(field_constants),2**n))  
  for i in range(len(field_constants)):
    H_dummy = qubit_operator_sparse(Honeycomb_Ising(H,n,h = field_constants[i],periodic_bc = False))
    m,g = sparse_magnetisation_expectation(n,H_dummy)
    magnetisations[i] = m
    g = g.reshape(2**n)

    Ground_states[i,:] = g 
  magnetisations = magnetisations.reshape(datapoints)
  return magnetisations,Ground_states,field_constants



### Generate HC data

In [ ]:
HC_magnetisations,HC_Ground_states,HC_field_constants = _get_data_honeycomb(ham,n,100)

In [ ]:
HC_magnetisations.shape

In [ ]:
HC_magnetisations,HC_Ground_states,HC_field_constants

### Classification

In [ ]:
HC_n = 13

classifier = MLP_2(HC_n,U_conv_6,pointwise_identity_embedding)
HC_gs = pd.DataFrame(HC_Ground_states)
HC_gs = pd.DataFrame(HC_gs.iloc[:,:])
HC_labels = np.zeros(len(HC_magnetisations))

for i in range(len(HC_magnetisations)):
  if HC_magnetisations[i] < -0.5:
    HC_labels[i] = 1
  else:
    HC_labels[i] = 0

HC_gs.insert(2**HC_n,'field_constants', HC_field_constants)
HC_gs.insert(2**HC_n,'labels',HC_labels)
HC_gs.insert(2**HC_n,'True Magnetisations', HC_magnetisations)
HC_X_train, HC_X_test, HC_y_train, HC_y_test = train_test_split(HC_gs.iloc[:,0:2**n], HC_gs.iloc[:,2**n:2**n+3], test_size=0.33, random_state=42)


In [ ]:
plt.plot(np.linspace(-20,40,100),HC_magnetisations)
#plt.xscale('log')
plt.legend(['Magnetisation'])
plt.title('Magnetisations of various field strengths')
plt.xlabel('Relative transverse field strength')
plt.ylabel('Magnetisation')